# Farm ponds identification pipeline: Applications
Before you start this notebook, make sure that you have
- Pretrained model configuration: ```config.pkl``` in the [parameters folder](../../output/parameters/)
- Tiles of the satellite image: ```tile_x_y.png``` in [train](../../data/train/) and [val](../../data/val/) folders
- Tiles of the masks: ```tile_x_y.png``` in [train_mask](../../data/train_mask/) and [val_mask](../../data/val_mask/) folders

 The tiles and the mask should follow the naming scheme: ```tile_x_y.png```, where x is the top left coordinates ```(x,y)``` of the image (in pixles). 

## Install the packages for the pipeline
Make sure you have the environment set up done, so that we can import the packages used in this notebook. Check out the [setup info](../../README.md). Traning the instance segmentation model requires ```torch``` and ```detectron2```, so here we need to install the libraries first. You can skip this step if you already installed ```detectron2``` in the previous steps.


In [ ]:
!python3 -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

## Import the libraries that are used in the pipeline
Part of the libraries and requirements used in this pipeline are based on Detectron2 documentation. You can find more details of the pre-train models from [Meta Research's Github repository](https://github.com/facebookresearch/detectron2)

In [ ]:
import torch, detectron2

# Setup detectron2 logger
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os, json, cv2, random, sys
import matplotlib.pyplot as plt
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer
from detectron2.data import transforms as T
from detectron2.data import DatasetMapper, build_detection_train_loader
import cloudpickle
from detectron2.config import CfgNode
from detectron2.utils.visualizer import ColorMode
from PIL import Image


TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
# Paths
ponds_root = os.path.dirname(os.path.dirname(os.getcwd())) 
if ponds_root not in sys.path:
    sys.path.append(ponds_root)

train_image_path = os.path.join(ponds_root, "data/train.png")  # Path to the input image
train_mask_path =  os.path.join(ponds_root,"data/train_mask.png")
train_folder =  os.path.join(ponds_root,"data/train/")  # Output folder for tiles
train_mask_folder =  os.path.join(ponds_root,"data/train_mask/")
train_not_used_folder =  os.path.join(ponds_root,"data/train_not_used/")
val_folder =  os.path.join(ponds_root,"data/val/")
val_mask_folder =   os.path.join(ponds_root,"data/val_mask/")
output_folder = os.path.join(ponds_root, "output/")
parameter_folder = os.path.join(output_folder, "parameters/")
model_folder = os.path.join(output_folder, "model/")
performance_folder = os.path.join(output_folder, "performance/")
inference_path = os.path.join(performance_folder, "inference.txt")

best_performance_path = os.path.join(performance_folder, "best_performance.txt")
test_image_path = os.path.join(ponds_root, "data/test.png")  
test_folder = os.path.join(ponds_root,"data/test/")
test_mask_folder = os.path.join(ponds_root,"data/test_mask/")
test_array_folder = os.path.join(ponds_root,"data/test_array/")


In [ ]:
from utils import preprocess as prep
from utils import helpers as hp
from detectron2.data.datasets import register_coco_instances
register_coco_instances("pond_train", {}, os.path.join(train_folder,"train.json"), train_folder)
register_coco_instances("pond_val", {}, os.path.join(val_folder,"val.json"), val_folder)
pond_metadata = MetadataCatalog.get("pond_train")

In [ ]:
# Usage example:
best_paths = hp.read_paths_from_file(best_performance_path)


## Load the pre-trained model



In [ ]:
cfg = hp.load_from_cloudpickle(best_paths[1])
cfg.MODEL.WEIGHTS = best_paths[0]  # path to the trained model 
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

## Test on Kadwanchi Data

This is a test on of of the imagery for Kadwanchi, India. You can swap the test image here to any other satellite image. You can also skip the next cell if you already have image tiles instead of the complete image. The resolution is 1024x1024 pixel for each tile. The Ground Sample Distance (GSD) for ths image is 30cm/px. We recommend the same resolution as the training set for the best prediction results.

In [ ]:
Image.MAX_IMAGE_PIXELS = None
test_image_path = os.path.join(ponds_root,"data/test.png")
test_folder = os.path.join(ponds_root,"data/test/")
min_image_size = 100000  # < 3800 is empty
tile_width, tile_height = 1024, 1024  # Tile dimensions

prep.divide_and_save_image(test_image_path, test_folder, tile_width, tile_height)
prep.filter_tiles_by_size(test_folder, min_image_size) 

### Predicting results

In [ ]:
from detectron2.utils.visualizer import ColorMode
from itertools import compress
from skimage import io, color, segmentation

os.makedirs(test_folder, exist_ok=True)
os.makedirs(test_mask_folder, exist_ok=True)
image_files = [file for file in os.listdir(test_folder) if file.lower().endswith(('.png'))]

def cropper(org_image_path, mask_array, out_file_name):
    num_instances = mask_array.shape[0]
    mask_array = np.moveaxis(mask_array, 0, -1)
    mask_array_instance = []
    img = cv2.imread(str(org_image_path))
    output = np.zeros_like(img)
    for i in range(num_instances):
        mask_array_instance.append(mask_array[:, :, i:(i+1)])
        output = np.where(mask_array_instance[i] == True, 255, output)
    im = Image.fromarray(output)
    im.save(out_file_name)


# Loop through each image file
for d in image_files:
    im_path = os.path.join(test_folder, d)
    im = cv2.imread(im_path)
    outputs = predictor(im)
    
    masks = outputs['instances'].pred_masks
    masks= masks.to('cpu').numpy()
    #num, h, w= masks.shape
    mask_path = os.path.join(test_mask_folder, d)
    cropper(im_path, masks, mask_path)
    

# Show five random samples
for d in random.sample(image_files, 5):
    im_path = os.path.join(test_folder, d)
    print(im_path)
    im = cv2.imread(im_path)
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=pond_metadata,
                   scale=0.5,
                   instance_mode=ColorMode.IMAGE_BW   
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.imshow(out.get_image()[:, :, ::-1])
    plt.show()



## Postprocessing

### Mosaic

This part of the code stiches all the tiles together based on their locations on the image. 
 - Input: The test mask folder
 - Output: A merged PNG

In [ ]:
import utils.mosaic as mosc
os.makedirs(test_mask_folder, exist_ok=True)
os.makedirs(test_array_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)
mosc.process_directory(test_mask_folder, test_array_folder)
sorted_filenames = mosc.load_and_sort_filenames(test_array_folder)
merged_array, failed_list = mosc.merge_tiles(sorted_filenames, test_array_folder, (14849, 20937))
merged_image_path = os.path.join(output_folder, "merged_predictions.png")
mosc.save_merged_image(merged_array, merged_image_path)
output_txt_path = os.path.join(output_folder, "tiles_failed_to_count.txt")
mosc.save_failed_list(failed_list, output_txt_path)

### Georeferencing

This cell will geolocate the farmponds and assign coordinates (langitude, longitude) to the prediction results. You can skip this cell if you use a GIS software such QGIS as to perform georeferencing manually. Our example below uses a QGIS georeferenced TIF to obtain the best geolocations for the ponds. Please see the georeferencing tutorial for QGIS here: [https://docs.qgis.org/3.34/en/docs/user_manual/working_with_raster/georeferencer.html](https://docs.qgis.org/3.34/en/docs/user_manual/working_with_raster/georeferencer.html)

 - Input: The merged_predictions.png, the longitude and latitude of the four corners of the image
 - Output: A merged_predictions.tif


In [ ]:
import utils.georeference as georef
# Configuration - Replace these with your actual file paths and coordinates
georef_input_png = os.path.join(output_folder, "merged_predictions.png")
output_geotiff = os.path.join(output_folder, "georeferenced.tif")
top_left_x, top_left_y = 75.97936000, 19.94643500  # NW corner: Longitude, Latitude
bottom_right_x, bottom_right_y = 76.02171400, 19.88961000  # SE corner: Longitude, Latitude

dst_ds = georef.create_geotiff_copy(georef_input_png, output_geotiff)
geotransform = georef.calculate_geotransform_parameters(dst_ds, top_left_x, top_left_y, bottom_right_x, bottom_right_y)
georef.assign_geotransform_and_projection(dst_ds, geotransform)


### Area Calculation

This cell estimates the surface area of each labeled pond on the satellite image and creates a spreadsheet (CSV)that includes the location and area. 

 - Input: The georeferenced.tif
 - Output: A CSV of the instance area estimates

In [ ]:
import utils.area_calculator as ac
ac_input_tif = os.path.join(output_folder, "georeferenced.tif")
ac_output_csv = os.path.join(output_folder, "area_estimate.csv")
ac_output_png = os.path.join(output_folder, "labeled_instances.png")

image, thresholded = ac.load_and_threshold_image(ac_input_tif)
contours = ac.find_contours(thresholded)
geotransform, transform = ac.setup_coordinate_transformation(ac_input_tif)
if geotransform and transform:
    data = ac.calculate_objects_data(contours, geotransform, transform)
    ac.save_data_and_image(data, image, ac_output_csv, ac_output_png)
else:
    print("Failed to perform coordinate transformation. Exiting.")